In [33]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [34]:
drivers = "../Resources/drivers.csv"
races = "../Resources/races.csv"
driver_standings = "../Resources/driver_standings.csv"
constructor_results = "../Resources/constructor_results.csv"
constructor_standings = "../Resources/constructor_standings.csv"
lap_times = "../Resources/lap_times.csv"
pit_stops = "../Resources/pit_stops.csv"
qualifying = "../Resources/qualifying.csv"
status = "../Resources/status.csv"


drivers_df = pd.read_csv(drivers)
races_df = pd.read_csv(races)
driver_standings_df= pd.read_csv(driver_standings)
constructor_results_df = pd.read_csv(constructor_results)
constructor_standings_df = pd.read_csv(constructor_standings)
lap_times_df = pd.read_csv(lap_times)
pit_stops_df = pd.read_csv(pit_stops)
qualifying_df = pd.read_csv(qualifying)
status_df = pd.read_csv(status)


In [28]:
drivers_df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [48]:
def calculate_age(born):
        born = datetime.datetime.strptime(born, "%m/%d/%Y").date()
        today = date.today()
        return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [73]:
age = pd.merge(drivers_df, results_df, on="driverId", how="inner")
age2 = age.drop(columns=['url'])
age2.head(261)

,driverId,driverRef,number_x,code,forename,surname,dob,nationality,resultId,raceId,...,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,1,18,...,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,27,19,...,5,4.0,56,+46.548,5525103,53,3,1:35.462,209.033,1
2,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,57,20,...,13,0.0,56,\N,\N,25,19,1:35.520,203.969,11
3,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,69,21,...,3,6.0,66,+4.187,5903238,20,3,1:22.017,204.323,1
4,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,90,22,...,2,8.0,58,+3.779,5213230,31,2,1:26.529,222.085,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,24746,1037,...,1,25.0,44,1:24:08.761,5048761,28,2,1:47.758,233.990,1
257,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,24772,1038,...,7,7.0,53,+17.245,6443301,34,1,1:22.746,252.033,1
258,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,24786,1039,...,1,26.0,59,2:19:35.060,8375060,58,1,1:18.833,239.518,1
259,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,24808,1040,...,3,15.0,53,+22.729,5663093,48,7,1:38.075,214.660,1


In [86]:
age_races = pd.merge(age2, races_df, on="raceId", how="inner")
age_races2 = age_races.drop(columns=['url', 'date'])
age_races2.head(25)

,driverId,driverRef,number_x,code,forename,surname,dob,nationality,resultId,raceId,...,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name,time_y
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,1,18,...,39,2,1:27.452,218.300,1,2008,1,1,Australian Grand Prix,04:30:00
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,2,18,...,41,3,1:27.739,217.586,1,2008,1,1,Australian Grand Prix,04:30:00
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,3,18,...,41,5,1:28.090,216.719,1,2008,1,1,Australian Grand Prix,04:30:00
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,4,18,...,58,7,1:28.603,215.464,1,2008,1,1,Australian Grand Prix,04:30:00
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,5,18,...,43,1,1:27.418,218.385,1,2008,1,1,Australian Grand Prix,04:30:00
5,6,nakajima,\N,NAK,Kazuki,Nakajima,1985-01-11,Japanese,6,18,...,50,14,1:29.639,212.974,11,2008,1,1,Australian Grand Prix,04:30:00
6,7,bourdais,\N,BOU,Sébastien,Bourdais,1979-02-28,French,7,18,...,22,12,1:29.534,213.224,5,2008,1,1,Australian Grand Prix,04:30:00
7,8,raikkonen,7,RAI,Kimi,Räikkönen,1979-10-17,Finnish,8,18,...,20,4,1:27.903,217.180,5,2008,1,1,Australian Grand Prix,04:30:00
8,9,kubica,88,KUB,Robert,Kubica,1984-12-07,Polish,9,18,...,15,9,1:28.753,215.100,4,2008,1,1,Australian Grand Prix,04:30:00
9,10,glock,\N,GLO,Timo,Glock,1982-03-18,German,10,18,...,23,13,1:29.558,213.166,3,2008,1,1,Australian Grand Prix,04:30:00


In [84]:
age_races2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24860 entries, 0 to 24859
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   driverId         24860 non-null  int64  
 1   driverRef        24860 non-null  object 
 2   number_x         24860 non-null  object 
 3   code             24860 non-null  object 
 4   forename         24860 non-null  object 
 5   surname          24860 non-null  object 
 6   dob              24860 non-null  object 
 7   nationality      24860 non-null  object 
 8   resultId         24860 non-null  int64  
 9   raceId           24860 non-null  int64  
 10  constructorId    24860 non-null  int64  
 11  number_y         24860 non-null  object 
 12  grid             24860 non-null  int64  
 13  position         24860 non-null  object 
 14  positionText     24860 non-null  object 
 15  positionOrder    24860 non-null  int64  
 16  points           24860 non-null  float64
 17  laps        

In [61]:
age2.columns

Index(['driverId', 'driverRef', 'number_x', 'code', 'forename', 'surname',
       'dob', 'nationality', 'resultId', 'raceId', 'constructorId', 'number_y',
       'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps',
       'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId'],
      dtype='object')

In [69]:
age2.driverRef.value_counts()

raikkonen             327
barrichello           326
alonso                314
button                309
michael_schumacher    308
                     ... 
botha                   1
apicella                1
goethals                1
rhodes                  1
dusio                   1
Name: driverRef, Length: 848, dtype: int64

In [29]:
races_df.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url'], dtype='object')

In [67]:
races_df.head(20)

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
5,6,2009,6,6,Monaco Grand Prix,2009-05-24,12:00:00,http://en.wikipedia.org/wiki/2009_Monaco_Grand...
6,7,2009,7,5,Turkish Grand Prix,2009-06-07,12:00:00,http://en.wikipedia.org/wiki/2009_Turkish_Gran...
7,8,2009,8,9,British Grand Prix,2009-06-21,12:00:00,http://en.wikipedia.org/wiki/2009_British_Gran...
8,9,2009,9,20,German Grand Prix,2009-07-12,12:00:00,http://en.wikipedia.org/wiki/2009_German_Grand...
9,10,2009,10,11,Hungarian Grand Prix,2009-07-26,12:00:00,http://en.wikipedia.org/wiki/2009_Hungarian_Gr...


In [31]:
driver_standings_df.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


In [32]:
results_df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [35]:
constructor_results_df.head()

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,\N
1,2,18,2,8.0,\N
2,3,18,3,9.0,\N
3,4,18,4,5.0,\N
4,5,18,5,2.0,\N


In [36]:
constructor_standings_df.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0


In [37]:
lap_times_df.head()

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [38]:
qualifying_df.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236


In [55]:
status_df

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine
...,...,...
131,134,+38 Laps
132,135,Brake duct
133,136,Seat
134,137,Damage


In [42]:
best_driver = pd.merge(drivers_df, qualifying_df, on="driverId", how="inner")
best_driver

,driverId,driverRef,number_x,code,forename,surname,dob,nationality,url,qualifyId,raceId,constructorId,number_y,position,q1,q2,q3
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1,18,1,22,1,1:26.572,1:25.187,1:26.714
1,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,26,19,1,22,4,1:35.392,1:34.627,1:36.709
2,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,47,20,1,22,3,1:32.750,1:31.922,1:33.292
3,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,71,21,1,22,5,1:21.366,1:20.825,1:22.096
4,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,91,22,1,22,3,1:26.192,1:26.477,1:27.923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,849,latifi,6,LAT,Nicholas,Latifi,1995-06-29,Canadian,http://en.wikipedia.org/wiki/Nicholas_Latifi,8537,1038,3,6,20,1:21.717,\N,\N
8590,849,latifi,6,LAT,Nicholas,Latifi,1995-06-29,Canadian,http://en.wikipedia.org/wiki/Nicholas_Latifi,8556,1039,3,6,19,1:17.320,\N,\N
8591,849,latifi,6,LAT,Nicholas,Latifi,1995-06-29,Canadian,http://en.wikipedia.org/wiki/Nicholas_Latifi,8576,1040,3,6,19,1:35.066,\N,\N
8592,849,latifi,6,LAT,Nicholas,Latifi,1995-06-29,Canadian,http://en.wikipedia.org/wiki/Nicholas_Latifi,8595,1041,3,6,18,1:27.812,\N,\N


In [46]:
pd.merge(constructor_results_df, constructor_standings_df, on="constructorId", how="outer")


,constructorResultsId,raceId_x,constructorId,points_x,status,constructorStandingsId,raceId_y,points_y,position,positionText,wins
0,1,18,1,14.0,\N,1.0,18.0,14.0,1.0,1,1.0
1,1,18,1,14.0,\N,7.0,19.0,24.0,1.0,1,1.0
2,1,18,1,14.0,\N,18.0,20.0,28.0,3.0,3,1.0
3,1,18,1,14.0,\N,29.0,21.0,34.0,3.0,3,1.0
4,1,18,1,14.0,\N,40.0,22.0,42.0,3.0,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4053568,16173,1042,213,10.0,\N,27557.0,1038.0,47.0,7.0,7,1.0
4053569,16173,1042,213,10.0,\N,27567.0,1039.0,53.0,7.0,7,1.0
4053570,16173,1042,213,10.0,\N,27587.0,1040.0,59.0,7.0,7,1.0
4053571,16173,1042,213,10.0,\N,27597.0,1041.0,67.0,7.0,7,1.0


In [ ]:
#def calculate_age(born):
        #born = datetime.datetime.strptime(born, "%m/%d/%Y").date()
        #today = date.today()
        #return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
        
#age = gb2
#age['age'] = age['dob'].apply(calculate_age)
#age['age_by_decade'] = pd.cut(x=age['age'], bins=[20,29,39,49,59,69,200], labels=['20s', '30s', '40s','50s','60s','70s +'])
#age['freq_age']=age.groupby(by='age_by_decade')['positionOrder'].transform('count').sort_values()
#age = age[age["freq"] >20 ]
#age2 = age.drop_duplicates(subset=['age_by_decade'])
#age2.dropna()
#age2.head()